# Build Model

In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

## Get Device for Training

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))

Using cuda device


## Define the Class

We define our neural network by subclassing nn.Module, and initialize the neural network layers in __init__. Every nn.Module subclass implements the operations on input data in the forward method.

In [3]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28 * 28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

We create an instance of NeuralNetwork, and move it to the device, and print it’s structure.

In [6]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
)


To use the model, we pass it the input data. This executes the model’s forward, along with some background operations. Do not call model.forward() directly!

Calling the model on the input returns a 10-dimensional tensor with raw predicted values for each class. We get the prediction probabilities by passing it through an instance of the nn.Softmax module.

In [8]:
X = torch.rand(1, 28, 28, device = device)
logits = model(X)
pred_probab = nn.Softmax(dim = 1)(logits)
y_pred = pred_probab.argmax(dim = 1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([0], device='cuda:0')


## Model Layers

Lets break down the layers in the FashionMNIST model. To illustrate it, we will take a sample minibatch of 3 images of size 28x28 and see what happens to it as we pass it through the network.

In [9]:
input_image = torch.rand(3, 28, 28)
print(input_image.size())

torch.Size([3, 28, 28])


## nn.Flatten

We initialize the nn.Flatten layer to convert each 2D 28x28 image into a contiguous array of 784 pixel values ( the minibatch dimension (at dim=0) is maintained).

In [10]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


## nn.Linear

The linear layer is a module that applies a linear transformation on the input using it’s stored weights and biases.

In [11]:
layer1 = nn.Linear(in_features = 28 * 28, out_features = 20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


## nn.ReLU

Non-linear activations are what create the complex mappings between the model’s inputs and outputs. They are applied after linear transformations to introduce nonlinearity, helping neural networks learn a wide variety of phenomena.

In this model, we use nn.ReLU between our linear layers, but there’s other activations to introduce non-linearity in your model.

In [12]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[ 0.0887,  0.0907,  0.1219, -0.1293,  0.1656, -0.2033, -0.1498, -0.8783,
          0.0690,  0.0427, -0.0793, -0.0579,  0.4557,  0.3569,  0.0615, -0.2663,
          0.3892, -0.1860,  0.1530,  0.4496],
        [ 0.0199, -0.1021,  0.5306, -0.2606,  0.1919,  0.1149, -0.3621, -0.5827,
         -0.0817,  0.5418,  0.0266,  0.4244,  0.3725,  0.4244, -0.0621, -0.1740,
          0.0774, -0.4505,  0.1583,  0.0091],
        [ 0.1892,  0.1099,  0.2180, -0.4099, -0.0068, -0.0672, -0.3302, -0.5009,
          0.5049,  0.4919, -0.1064, -0.2001,  0.4931, -0.0678, -0.0730, -0.0567,
          0.2026, -0.3046, -0.0156,  0.3335]], grad_fn=<AddmmBackward>)


After ReLU: tensor([[0.0887, 0.0907, 0.1219, 0.0000, 0.1656, 0.0000, 0.0000, 0.0000, 0.0690,
         0.0427, 0.0000, 0.0000, 0.4557, 0.3569, 0.0615, 0.0000, 0.3892, 0.0000,
         0.1530, 0.4496],
        [0.0199, 0.0000, 0.5306, 0.0000, 0.1919, 0.1149, 0.0000, 0.0000, 0.0000,
         0.5418, 0.0266, 0.4244, 0.3725, 0.4244, 0.000

## nn.Sequential

nn.Sequential is an ordered container of modules. The data is passed through all the modules in the same order as defined. You can use sequential containers to put together a quick network like seq_modules.

In [13]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3, 28, 28)
logits = seq_modules(input_image)

## nn.Softmax

The last linear layer of the neural network returns logits - raw values in [-infty, infty] - which are passed to the nn.Softmax module. The logits are scaled to values [0, 1] representing the model’s predicted probabilities for each class. dim parameter indicates the dimension along which the values must sum to 1.

In [14]:
softmax = nn.Softmax(dim = 1)
pred_probab = softmax(logits)

## Model Parameters

Many layers inside a neural network are parameterized, i.e. have associated weights and biases that are optimized during training. Subclassing nn.Module automatically tracks all fields defined inside your model object, and makes all parameters accessible using your model’s parameters() or named_parameters() methods.

In this example, we iterate over each parameter, and print its size and a preview of its values.

In [15]:
print("Model structure: ", model, "\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure:  NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
) 


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[-0.0157,  0.0205,  0.0223,  ..., -0.0336, -0.0303, -0.0102],
        [ 0.0357, -0.0198,  0.0082,  ..., -0.0029,  0.0052, -0.0084]],
       device='cuda:0', grad_fn=<SliceBackward>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([0.0226, 0.0044], device='cuda:0', grad_fn=<SliceBackward>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[-0.0429, -0.0214, -0.0116,  ..., -0.0123,  0.0083, -0.0320],
        [ 0.0065, -0.0092,  0.0255,  ..., -0.0195,  0.0258,  0.0057]],
       device='cuda:0